In [1]:
import numpy as np
import os
import re

import multiprocess as mp
import warnings
warnings.filterwarnings("ignore")

import plotly.graph_objs as go

from trProcess import *

In [2]:
def ProcessBaseline (conf):
    exps, expconf, tracenum, indir, name, baseline = conf
    apps = expconf.split("_")
    p90Fct, maxAppA, maxAppB, jfi = [], [], [], []
    for exp in exps:
        if not re.search (expconf, exp):
            continue
        p90FctTrace, maxAppATrace, maxAppBTrace, jfiTrace = [], [], [], []
        for traceid in range(tracenum):
            trace = ""
            conf = os.path.join (indir, exp, baseline, trace)
                
            bws = []
            curMaxApp, curAppDur, _, _ = appTraceProcessor (os.path.join (conf, 'app.tr'), startTime=14000 + 13000 * apps[0].count("f"))
            curMaxAppFlows = 0
            for port, appDelay in curAppDur.items():
                if port in range(10000, 20000):
                    curMaxAppFlows = max(curMaxAppFlows, appDelay)
            maxAppATrace.append(curMaxAppFlows)
    
            curAvgFct, curP90Fct, fctLastStart, fctFirstEnd, fcts = fctTraceProcessor(os.path.join (conf, 'fct.tr'))
            p90FctTrace.append(curP90Fct)
            bws = np.array(bws)
            jfiTrace.append(pow(np.nansum(bws),2) / (bws.size * np.nansum(np.square(bws))))
        p90Fct.append(np.average(p90FctTrace))
        maxAppA.append(np.average(maxAppATrace))
        jfi.append(np.average(jfiTrace))
    print ("%d\t%d\t%s" % (np.average (maxAppA), np.average (p90Fct), name))
    # return go.Scatter(
    #     x=[np.average (maxAppA)],
    #     y=[np.average (p90Fct)],
    #     mode="markers+text",
    #     text=name,
    #     name=name,
    #     textposition="bottom center"
    # )

In [10]:
tracenum = 1
baselinelist = {
    "FQ": "FqCoDel_0_500ms1000", 
    "FIFO": "PfifoFast_0_1000",
    "Confucius": "Auto_0_1.00_ExpClass4.00",
    "CBQ (1:1)": "Dwrr_9_1.00",
}

expconfs = [
    "^46_Video1_Web102.*000_20Mbps_1000Mbps_1000Mbps$",
    # "^46_Video1_Web[100].*000_1000Mbps_20Mbps_1000Mbps$",
    # "^46_Video1_Web[100].*000_1000Mbps_1000Mbps_20Mbps$",
]
indir = "ns-3.34/logs/"
exps = os.listdir (indir)
for expconf in expconfs:
    pool = mp.Pool (1)
    pool.map (ProcessBaseline, [(exps, expconf, tracenum, indir, name, baseline) for name, baseline in baselinelist.items()])
    pool.close ()
    pool.join ()
    # data1 = list (map (ProcessBaseline, [(exps, expconf, tracenum, indir, name, baseline) for name, baseline in baselinelist.items()]))

    # fig = go.Figure (data=data1)
    # fig.update_layout(
    #     title=expconf,
    #     xaxis_title="Video Latency Jitter (ms)",
    #     yaxis_title="Page Loading Time (ms)"
    # )
    # fig.show()

60	454	FQ
60	454	FIFO
60	454	Confucius
60	454	CBQ (1:1)
